In [1]:
from numba import njit, literal_unroll
import numpy as np
from quantfreedom.utils.caching import clear_cache
# clear_cache()

In [2]:
cart_array_dt = np.dtype([
    ('tester', np.float_),
    ('two', np.float_),
], align=True)


@njit(cache=True)
def test():
    arrays = (
        np.array([1., 2., 3.]),
        np.array([np.nan, 6.]),
    )
    dtype_names = (
        'tester',
        'two',
    )

    n = 1
    for x in arrays:
        n *= x.size
    out = np.zeros((n, len(arrays)), dtype=np.float_)
    cart_array = np.zeros(n, dtype=cart_array_dt)

    for i in range(len(arrays)):
        m = int(n / arrays[i].size)
        out[:n, i] = np.repeat(arrays[i], m)
        n //= arrays[i].size

    n = arrays[-1].size
    for k in range(len(arrays)-2, -1, -1):
        n *= arrays[k].size
        m = int(n / arrays[k].size)
        for j in range(1, arrays[k].size):
            out[j*m:(j+1)*m, k+1:] = out[0:m, k+1:]

    # literal unroll
    counter = 0
    for dtype_name in literal_unroll(dtype_names):
        for col in range(n):
            cart_array[dtype_name][col] = out[col][counter]
        counter += 1
    return cart_array


test()


array([(1., nan), (1.,  6.), (2., nan), (2.,  6.), (3., nan), (3.,  6.)],
      dtype={'names': ['tester', 'two'], 'formats': ['<f8', '<f8'], 'offsets': [0, 8], 'itemsize': 16, 'aligned': True})

In [3]:
from numba import njit, literal_unroll
import numpy as np
cart_array_dt = np.dtype([
    ('order_settings_id', np.float_),
    ('leverage', np.float_),
], align=True)

@njit(cache=True)
def test():
    arrays = (
        np.array([0.]),
        np.array([1., 2., 3.]),
    )

    dtype_names = (
        'order_settings_id',
        'leverage',
    )

    n = 1
    for x in arrays:
        n *= x.size
    out = np.empty((n, len(arrays)))
    cart_array = np.empty(n, dtype=cart_array_dt)

    for i in range(len(arrays)):
        m = int(n / arrays[i].size)
        out[:n, i] = np.repeat(arrays[i], m)
        n //= arrays[i].size

    n = arrays[-1].size
    for k in range(len(arrays)-2, -1, -1):
        n *= arrays[k].size
        m = int(n / arrays[k].size)
        for j in range(1, arrays[k].size):
            out[j*m:(j+1)*m, k+1:] = out[0:m, k+1:]
    
    # literal unroll
    counter = 0
    for dtype_name in literal_unroll(dtype_names):
        for col in range(n):
            cart_array[dtype_name][col] = out[col][counter]
        counter += 1
    return cart_array
test()


array([(0., 1.), (0., 2.), (0., 3.)],
      dtype={'names': ['order_settings_id', 'leverage'], 'formats': ['<f8', '<f8'], 'offsets': [0, 8], 'itemsize': 16, 'aligned': True})

In [4]:
clear_cache()